## Генератор человеко-читаемых дат на основе RNN и посимвольной генерации

In [1]:
import numpy as np
from utils import *
import random
import pprint

from faker import Faker
from tqdm import tqdm
from babel.dates import format_date

### Генерация обучающей выборки

In [2]:
fake = Faker()
Faker.seed(12345)
random.seed(12345)

# Define format of the data we would like to generate
FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

# change this if you want it to work with another language
LOCALES = ['en_US']

def load_date():
    """
        Loads some fake dates 
        :returns: tuple containing human readable string, machine readable string, and date object
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS),  locale='en_US') # locale=random.choice(LOCALES))
        human_readable = human_readable.lower()
        human_readable = human_readable.replace(',','')
        machine_readable = dt.isoformat()
        
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

def load_dataset(m):
    """
        Loads a dataset with m examples and vocabularies
        :m: the number of examples to generate
    """
    
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    Tx = 30
    

    for i in tqdm(range(m)):
        h, m, _ = load_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

In [3]:
m = 100000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████| 100000/100000 [00:03<00:00, 29078.41it/s]


In [4]:
# Формирование обучающей выборки
d = np.array(dataset)
data = '\n'.join(d[:, 0])
examples = d[:, 0]

# Составление словаря токенов - a-z, \n
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 1684573 total characters and 36 unique characters in your data.


Выведем список всех используемых символов.

In [5]:
chars = sorted(chars)
print(chars)

['\n', ' ', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


Отображение символов на числовые индексы и обратно

In [6]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

{   0: '\n',
    1: ' ',
    2: '.',
    3: '/',
    4: '0',
    5: '1',
    6: '2',
    7: '3',
    8: '4',
    9: '5',
    10: '6',
    11: '7',
    12: '8',
    13: '9',
    14: 'a',
    15: 'b',
    16: 'c',
    17: 'd',
    18: 'e',
    19: 'f',
    20: 'g',
    21: 'h',
    22: 'i',
    23: 'j',
    24: 'l',
    25: 'm',
    26: 'n',
    27: 'o',
    28: 'p',
    29: 'r',
    30: 's',
    31: 't',
    32: 'u',
    33: 'v',
    34: 'w',
    35: 'y'}


Усечение (ограничение по модулю) градиентов во избежание "градиентного взрыва".

In [7]:
def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

Реализация ячейки RNN и сэмплирование следующего символа в соответствии с распределением вероятностей, полученным на выходе RNN

In [8]:
def sample(parameters, char_to_ix, seed):
    """
    parameters -- весовые матрицы и векторы смещения Waa, Wax, Wya, by, and b. 
    char_to_ix -- отображение символов на индексы.

    Returns:
    indices -- индексы сэмплированных символов.
    """
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Значение первого входного вектора принимается нулевым
    x = np.zeros((vocab_size, 1))
    # Значение скрытого состояния в начале - нулевое
    a_prev = np.zeros((n_a, 1))
    
    # Массив для хранения индексов сгенерированных символов
    indices = []
    
    # Текущий индекс последнего сгенерированного символа
    idx = -1 
    
    # Реализация работы RNN и сохранение результатов в массив indices 
    # Максимальная длина выходной последовательности - 50 символов 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while idx != newline_character and counter != 50:
        
        # Уравнения прямого прохода
        a = np.tanh(Waa @ a_prev + Wax @ x + b)
        z = Wya @ a + by
        y = softmax(z) 
        
        # Сэмплирование из распределения вероятностей y
        idx = np.random.choice(range(len(np.ravel(y))), p = np.ravel(y))

        # Дописываем полученный индекс в indices
        indices.append(idx)
        
        # Подготавливаем вход для следующего временного шага (равен выходу на текущем)
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        # Сохраняем скрытое состояние
        a_prev = a
        
        counter +=1

    if counter == 50:
        indices.append(char_to_ix['\n'])
    
    return indices

Реализация одного цикла работы модели - прямой проход, вычисление градиента, обновление весов

In [9]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    X -- элемент обучающей выборки.
    Y = X, сдвинутый на 1 позицию влево
    a_prev -- предыдущее скрытое состояние.
    parameters:
        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
        b --  Bias, numpy array of shape (n_a, 1)
        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate
    
    Returns:
    loss -- функция потерь (кросс-энтропия)
    gradients -- градиенты матриц и векторов смещения:
        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
        db -- Gradients of bias vector, of shape (n_a, 1)
        dby -- Gradients of output bias vector, of shape (n_y, 1)
    """
    
    # Прямой проход
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Обратный проход
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Усечение градиентов
    gradients = clip(gradients, 5)
    
    # Обновление параметров (градиентный шаг)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    
    return loss, gradients, a[len(X)-1]

Обучение модели на обучающей выборке.
Используется стохастический градиентный спуск, каждые 2000 итераций выводятся несколько примеров генерации названий.

In [10]:
def model(ix_to_char, char_to_ix, num_iterations = 20000, n_a = 50, dino_names = 7, vocab_size = 36, verbose = False):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text (size of the vocabulary)
    
    Returns:
    parameters -- learned parameters
    """
    
    # Инициализация параметров
    n_x, n_y = vocab_size, vocab_size
    parameters = initialize_parameters(n_a, n_x, n_y)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Перемешивание обучающей выборки
    np.random.shuffle(examples)
    
    # Инициализация начального состояния
    a_prev = np.zeros((n_a, 1))
    
    # Главный цикл оптимизации
    for j in range(num_iterations):
        
        # Выбираем элемент обучающей выборки
        idx = j % len(examples)
        
        # Подаем его на вход, добавляем в начало токен None - сигнал начала генерации
        single_example = examples[idx]
        single_example_chars = [c for c in single_example]
        single_example_ix = [char_to_ix[c] for c in single_example_chars]
        X = [None] + single_example_ix
        
        # Подаем сдвинутую влево на 1 позицию последовательность на выход
        ix_newline = char_to_ix['\n']
        Y = X[1:] + [ix_newline]

        # Один шаг оптимизации
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)

        # Выводим несколько примеров генерации названий по мере обучения
        if j % 2000 == 0:
            print('Iteration: %d' % j + '\n')
            for name in range(dino_names):
                # Сэмплируем несколько выходов нейросети
                sampled_indices = sample(parameters, char_to_ix, seed=42)
                print_sample(sampled_indices, ix_to_char)
      
            print('\n')
        
    return parameters

Проверяем работу модели. По мере обучения качество генерации значительно улучшается.

In [12]:
parameters = model(ix_to_char, char_to_ix, num_iterations=200000, verbose = True)

Iteration: 0

7000 t44rdei/1u.2pc1tyeu86m5b2ge0m3743m77t9u53l
Pgu8ea8o a6fs7/wwva5ncif9.m6boy9bv
I39wh
N2jr vf
 71ja v30li9yb4 4f7n7cu9gubm7.u3fej8pty10w6up
Hh2y1rou5.b8h6/wucmmeel4mymlr8afi8a0
P1i/gm/nv078.4lydyi3uy evw4/c19w1o/j./p3bwaufnp.p5


Iteration: 2000

Riday july 19 1982
Sovtary optember 27 2919
3 267 23 1976
Sonday jhn3 24 2015
1 2019
V maril 2072
Day jany 2 16 5atuul 2 2020


Iteration: 4000

11
June 2020
Sepr32922
Ariepdec 2001
T 20 1972
Ter 1005
T 21 2015


Iteration: 6000

Thur 7 2 1980
T fet 1972
Friday august 7 1981
1 1979
Mocd ber 2008
Margh 24 20
Marril 1012


Iteration: 8000

Augest 15 1979
May 18 1991
Wednesday run 1991
6 1979
September 1995
Moce 1994
September 1982


Iteration: 10000

19 05 1983
Arcest 17 1998
Sapr a 2012
December 14 2006
September 30 1989
September 11 2002
5 2019


Iteration: 12000

Ril 1978
September 18 1972
17 februly 1975
November 1992
Sepbember 4 1988
3 sep 2009
Sunday september 23 2013


Iteration: 14000

Decembey 12 1999
Jan 3 2006
5 2006
